In [79]:
import numpy as np
from math import ceil
import matplotlib.pyplot as plt
import sys
import re
import pandas as pd
import torch
import torch.nn as nn
import scipy
from scipy import ndimage
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from jupyterthemes import jtplot
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error, log_loss, hinge_loss, accuracy_score
import random
import cv2
from torchvision import models
from tqdm import tqdm

data = pd.read_csv('../../fer2013/fer2013.csv')

In [2]:
print(len(data))
print(len(data.loc[0, 'pixels'].split(' ')))
print(data.groupby('Usage').count()[['emotions']])

#Frequency of each label
print('0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral')
table = data.groupby('emotions').count()[['Usage']]
table['Pct'] = table['Usage']/table['Usage'].sum()
table['Pct'] = table['Pct'].map(lambda x: round(x, 3)*100)
table

35887
2304
             emotions
Usage                
PrivateTest      3589
PublicTest       3589
Training        28709
0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral


,Usage,Pct
emotions,,
0,4953,13.8
1,547,1.5
2,5121,14.3
3,8989,25.0
4,6077,16.9
5,4002,11.2
6,6198,17.3


## 1) Converting to numpy datasets

In [3]:
def convert_to_numpy(data):
    X = data[:,1]
    X = np.asarray([np.asarray(X[i].split(" ")) for i in range(X.shape[0])])
    X = np.asarray([X[i].reshape(48,48).astype(int) for i in range(X.shape[0])])
    y = data[:,0]
    return (X,y)

In [4]:
train = data[data['Usage'] == 'Training']
train_X, train_Y = convert_to_numpy(train.values)
valid = data[data['Usage'] == 'PrivateTest']
valid_X, valid_Y = convert_to_numpy(valid.values)
test = data[data['Usage'] == 'PublicTest']
test_X, test_Y = convert_to_numpy(test.values)

## 2) Function to extract the batches from the dataset

In [5]:
def data_iter(x, y, batch_size):
    dataset_size = x.shape[0]
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            break   
        batch_indices = order[start:start + batch_size]
        yield np.asarray([x[index] for index in batch_indices]) ,np.asarray([y[index] for index in batch_indices])

## 3) Further data preprocessing

#### Zoom the image and convert 1 channel to 3 channel dataset

In [7]:
def to_rgb1a(data, w, h):

    R, _, _ = data.shape
    temp = np.zeros((R, 3, w, h))
    
    for i in tqdm(range(R)):
        im = data[0]
        ret = np.empty((3, w, h), dtype=np.uint8)
        im = cv2.resize(im.astype(float), (w, h), interpolation=cv2.INTER_LINEAR)
        ret[0, :, :] =  ret[1, :, :] =  ret[2, :, :] =  im
        temp[i] = ret
        data = np.delete(data, 0, 0)
    
    return temp

## 4) Training stage setup

In [6]:
def early_stop(val_acc_history, t=2, required_progress=0.001):    
    cnt = 0 # initialize the count --> to store count of cases where difference in
                                    #  accuracy is less than required progress.
    
    if(len(val_acc_history) > 0): # if list has size > 0 
        for i in range(t): # start the loop
            index = len(val_acc_history) - (i+1) # start from the last term in list and move to the left
            if (index >= 1): # to check if index != 0 --> else we can't compare to previous value
                if (abs(val_acc_history[index] - val_acc_history[index-1]) < required_progress):
                    cnt += 1 # increase the count value
                else:
                    break # break if difference is grea-ter 
    
    if(cnt != t): # if count is equal to t, return True
        return False
    else:
        return True
    

def train(train_X, train_Y, valid_X, valid_Y, optimizer, model, batch_size, num_epochs, criterion, to_Add_Softmax=False, is_inception=False):
    losses = []
    total_batches = int(train_X.shape[0]/ batch_size)
    validation_losses = []
    
    eval_every = 10
    print_every = 10
    validate_every = int((eval_every/100)*total_batches)
    show_every = int((print_every/100)*total_batches)
    
    for epoch in range(1, num_epochs+1):
        stop_training = False
        train_data = data_iter(train_X, train_Y, batch_size)
        for i, (x,y) in enumerate(train_data):
            x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
            y = Variable(torch.from_numpy(y).type(torch.LongTensor))
            model.train(True)
            optimizer.zero_grad()
            outputs = model(x)
            if is_inception == True:
                outputs = outputs[0]
            if to_Add_Softmax == True:
                outputs = nn.functional.softmax(outputs)
            loss = criterion(outputs, y)
            losses.append(loss.data[0])
            loss.backward()


            optimizer.step()
            
            if (i+1)%validate_every == 0:
                valid_loss_temp = []
                valid_data = data_iter(valid_X, valid_Y, batch_size)
                for j, (v_x, v_y) in enumerate(valid_data):
                    v_x = Variable(torch.from_numpy(v_x).type(torch.FloatTensor))
                    v_y = Variable(torch.from_numpy(v_y).type(torch.LongTensor))
                    model.eval()
                    val_outputs = model(v_x)
                    eval_loss = criterion(val_outputs, v_y)
                    valid_loss_temp.append(eval_loss.data[0])
                validation_losses.append(np.mean(valid_loss_temp))
                stop_training = early_stop(validation_losses, 3)
                
            if stop_training:
                print("earily stop triggered")
                break
            if (i+1) % show_every == 0:
                print('Epoch: [{0}/{1}], Step: [{2}/{3}], Train loss: {4}, Validation loss:{5}'.format(
                           epoch, num_epochs, i+1, total_batches, np.mean(losses)/(total_batches*epoch), np.mean(np.array(validation_losses))))
        if stop_training == True:
            break

## 5) Removing last layer & extracting features

In [32]:
def remove_last_layer(model):
    mod = list(model.classifier.children())
    mod.pop()
    new_classifier = torch.nn.Sequential(*mod)
    model.classifier = new_classifier
    return model

In [35]:
def extract_features(data, model):
    model = remove_last_layer(model)
    return model(Variable(torch.from_numpy(data).type(torch.FloatTensor)))

## 6) Training the model

#### Experimenting with a smaller dataset (as running imagenet architectures locally is very time consuming)

In [17]:
old_train_X = train_X
train_X = to_rgb1a(old_train_X[0:1000], 227, 227)
old_valid_X = valid_X
valid_X = to_rgb1a(old_valid_X[0:500], 227, 227)
old_test_X = test_X
test_X = to_rgb1a(old_test_X[0:500], 227, 227)

100%|██████████| 500/500 [00:01<00:00, 357.26it/s]


## 6.1) Only ImageNet Architectures

### 1) first model - VGG (Note that VGG is the very slow among everyone else) 

In [ ]:
num_labels = 7
num_epochs = 5
learning_rate = 0.01
kernel_size = 3
batch_size = 80

criterion = nn.CrossEntropyLoss()

vgg = models.vgg16(pretrained=True)
num_ftrs = vgg.classifier

## Modifying and training only the last layer
for param in vgg.parameters():
    param.requires_grad = False

vgg.classifier._modules['6'] = nn.Linear(4096, num_labels)
for param in vgg.classifier[6].parameters():
    param.requires_grad = True
    
optimizer = optim.Adam(vgg.classifier[6].parameters(), lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, vgg, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

#### 2.2) Alexnet

In [19]:
### Still some modification needs to be made

alexnet = models.alexnet(pretrained=True)
num_ftrs = alexnet.classifier

## Modifying and training only the last layer
for param in alexnet.parameters():
    param.requires_grad = False

alexnet.classifier._modules['6'] = nn.Linear(4096, num_labels)
# num_ftrs = alexnet.classifier[6].in_features
# alexnet.classifier[6].out_features = num_labels
for param in alexnet.classifier[6].parameters():
    param.requires_grad = True
    
#optimizer = optim.Adam(filter(lambda p: p.requires_grad, alexnet.parameters()), alexnet.classifier.parameters())
optimizer = optim.Adam(alexnet.classifier[6].parameters(), lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, alexnet, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel\__main__.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: [1/1], Step: [1/12], Train loss: 0.1632013519605001, Validation loss:11.684929211934408
Epoch: [1/1], Step: [2/12], Train loss: 0.1643969863653183, Validation loss:11.655517816543579
Epoch: [1/1], Step: [3/12], Train loss: 0.1654719743463728, Validation loss:11.722200711568197
Epoch: [1/1], Step: [4/12], Train loss: 0.16535725692907968, Validation loss:11.647497693697611
Epoch: [1/1], Step: [5/12], Train loss: 0.16612525383631388, Validation loss:11.563029861450195
Epoch: [1/1], Step: [6/12], Train loss: 0.16569271352556017, Validation loss:11.443655490875244
Epoch: [1/1], Step: [7/12], Train loss: 0.16496338163103377, Validation loss:11.379524321783157
Epoch: [1/1], Step: [8/12], Train loss: 0.1645751123627027, Validation loss:11.320532818635304
Epoch: [1/1], Step: [9/12], Train loss: 0.16432406725706877, Validation loss:11.249501228332518
Epoch: [1/1], Step: [10/12], Train loss: 0.1649247646331787, Validation loss:11.187508185704548
Epoch: [1/1], Step: [11/12], Train loss: 0.1

#### 3) resnet

In [93]:
resnet = models.resnet50(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 7 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, num_labels)
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, resnet, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel\__main__.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: [1/1], Step: [1/12], Train loss: 0.16207279761632284, Validation loss:3.240700642267863
Epoch: [1/1], Step: [2/12], Train loss: 0.1620688041051229, Validation loss:2.753064215183258
Epoch: [1/1], Step: [3/12], Train loss: 0.16196048922008938, Validation loss:2.516954706774818
Epoch: [1/1], Step: [4/12], Train loss: 0.16179496298233667, Validation loss:2.3845895628134413
Epoch: [1/1], Step: [5/12], Train loss: 0.16150825421015422, Validation loss:2.296902922789256
Epoch: [1/1], Step: [6/12], Train loss: 0.16132187015480465, Validation loss:2.233507063653734
Epoch: [1/1], Step: [7/12], Train loss: 0.16137162418592543, Validation loss:2.1852992177009587
Epoch: [1/1], Step: [8/12], Train loss: 0.16126281271378198, Validation loss:2.1471483260393143
Epoch: [1/1], Step: [9/12], Train loss: 0.16128376567805255, Validation loss:2.117350732838666
Epoch: [1/1], Step: [10/12], Train loss: 0.16130946079889932, Validation loss:2.09268914659818
Epoch: [1/1], Step: [11/12], Train loss: 0.16119

#### 4) Inception

##### Inception requires input size to be (299, 299)

In [29]:
train_X = to_rgb1a(old_train_X[0:1000], 299, 299)
valid_X = to_rgb1a(old_valid_X[0:500], 299, 299)
test_X = to_rgb1a(old_test_X[0:500], 299, 299)

100%|██████████| 500/500 [00:01<00:00, 302.43it/s]


In [ ]:
incptn = models.inception_v3(pretrained=True)
# freeze all model parameters
for param in incptn.parameters():
    param.requires_grad = False

# new final layer with 7 classes
num_ftrs = incptn.fc.in_features
incptn.fc = torch.nn.Linear(num_ftrs, num_labels)
optimizer = optim.Adam(incptn.fc.parameters(), lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, incptn, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=True)

## 6.2) Ensemble of ImageNet and LR

### Multi class accuracy


In [81]:
def multi_class_accuracy(pred, true):
    return sum(pred == true)/pred.shape[0]

### 1) Alexnet + LR

In [92]:
# get features for train, valid and test data
alexnet = models.alexnet(pretrained=True)
features_train = extract_features(train_X, alexnet)
features_train = np.array(features_train.data)
features_valid = extract_features(valid_X, alexnet)
features_valid = np.array(features_valid.data)
features_test = extract_features(test_X, alexnet)
features_test = np.array(features_test.data)

# Use both train and validation for training SVM
features_train = np.concatenate((features_train, features_valid))
train_Y = np.concatenate((train_Y[:1000], valid_Y[:500]))

# Use sklearn for svm
clf = svm.SVC(C=2, kernel='rbf', decision_function_shape='ovr')
clf.fit(features_train, train_Y.astype(int))

# Prediction on test set
pred_y = clf.predict(features_test)
test_accuracy = multi_class_accuracy(pred_y, test_Y[:500])
print(test_accuracy)

0.232


## 7) Calculating accuracy on test set

In [30]:
#test_output = model(Variable(torch.from_numpy(test_X).type(torch.FloatTensor)))
resnet.train(False)
test_output = resnet(Variable(torch.from_numpy(test_X).type(torch.FloatTensor)))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
accuracy = sum(pred_y == test_Y[0:500])/len(test_Y[0:500])
print(accuracy)

0.156
